# Andre Marroquin 22266
# Sergio orellana 221122
# Carlos Valladares
# Rodrigo Mansilla

#  Preparación del entorno y carga de datos

In [240]:
import pandas as pd
import numpy as np
import unicodedata

# Cargar el archivo Excel
df = pd.read_excel('Establecimientos.xlsx')

# Vista rápida
print(f"Dimensiones: {df.shape[0]} filas, {df.shape[1]} columnas")
df.head(5)

Dimensiones: 6590 filas, 18 columnas


,CODIGO,Unnamed: 1,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,16-01-0138-46,NaN,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
1,16-01-0139-46,NaN,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
2,16-01-0140-46,NaN,16-031,ALTA VERAPAZ,COBAN,"COLEGIO ""LA INMACULADA""",7A. AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
3,16-01-0141-46,NaN,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNÁNDEZ,HÉCTOR ROLANDO CHUN POOU,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
4,16-01-0142-46,NaN,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE 'EMILIO ROSAL...,3A AVE 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNÁNDEZ,VICTOR HUGO DOMÍNGUEZ REYES,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),ALTA VERAPAZ


# Descripción inicial del conjunto de datos

Filas iniciales: 6 590

Variables (columnas): 18

Nombres de columnas: ['CODIGO', 'Unnamed: 1', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO',
 'ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR',
 'NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN',
 'DEPARTAMENTAL']


In [241]:
# Conteo de valores faltantes por columna
missing = df.isna().sum().sort_values(ascending=False)
print("Valores faltantes por columna:\n", missing, "\n")



Valores faltantes por columna:
 Unnamed: 1         6590
TELEFONO             45
DIRECTOR             25
DIRECCION             2
SECTOR                0
PLAN                  0
JORNADA               0
MODALIDAD             0
STATUS                0
AREA                  0
CODIGO                0
NIVEL                 0
SUPERVISOR            0
ESTABLECIMIENTO       0
MUNICIPIO             0
DEPARTAMENTO          0
DISTRITO              0
DEPARTAMENTAL         0
dtype: int64 



In [242]:
# Verificación puntual
print(f"Unnamed: 1 faltantes: {missing['Unnamed: 1']}")
print(f"DIRECCION   faltantes: {missing['DIRECCION']}")
print(f"TELEFONO    faltantes: {missing['TELEFONO']}")
print(f"DIRECTOR    faltantes: {missing['DIRECTOR']}\n")

Unnamed: 1 faltantes: 6590
DIRECCION   faltantes: 2
TELEFONO    faltantes: 45
DIRECTOR    faltantes: 25



In [243]:
# Columnas constantes 
# Vacías todas las filas NaN
# Con un único valor
n_uniques = df.nunique(dropna=False)
const_empty = n_uniques[n_uniques == 0].index.tolist()
const_one   = n_uniques[n_uniques == 1].index.tolist()

print("Columnas constantes:")
print("Columnas completamente vacías:", const_empty)
print("Columnas con un único valor:", const_one, "\n")

Columnas constantes:
Columnas completamente vacías: []
Columnas con un único valor: ['Unnamed: 1', 'NIVEL', 'STATUS'] 



In [244]:
# Variables con más necesidad de limpieza
top_missing = missing.head(5).index.tolist()
text_cols = df.select_dtypes(include=['object']).columns
cardinalities = df[text_cols].nunique().sort_values(ascending=False)
top_card = cardinalities.head(5).index.tolist()

print("Variables con más necesidad de limpieza:")
print("Top 5 columnas con más faltantes:", top_missing)
print("Top 5 columnas de texto con más categorías únicas:", top_card)


Variables con más necesidad de limpieza:
Top 5 columnas con más faltantes: ['Unnamed: 1', 'TELEFONO', 'DIRECTOR', 'DIRECCION', 'SECTOR']
Top 5 columnas de texto con más categorías únicas: ['CODIGO', 'DIRECCION', 'TELEFONO', 'DIRECTOR', 'ESTABLECIMIENTO']


- Unnamed: 1 Columna vacía: debe eliminarse.
- ESTABLECIMIENTO Texto libre con mayúsculas/minúsculas inconsistentes, duplicados, errores ortográficos.
- DIRECCION	Espacios en blanco, caracteres especiales, faltantes, inconsistencias en formato.
- TELEFONO	Formatos variados guiones, paréntesis, espacios, valores faltantes, caracteres no numéricos.
- SUPERVISOR Nombres con mayúsculas/minúsculas mixtas, posibles nulos ocultos.
- DIRECTOR	Mismos problemas que SUPERVISOR, además de 25 faltantes.
- DISTRITO/MUNICIPIO Nombres con variantes ortográficas, acentos, mayúsculas.

# Limpieza de datos
## Estrategia de limpieza 
### obtener un DataFrame consistente eliminando columnas inútiles, homogeneizar texto y tratar valores faltantes de forma sencilla


In [245]:
# Eliminar columnas inutiles
cols_drop = ['Unnamed: 1', 'NIVEL', 'STATUS']
df1 = df.drop(columns=cols_drop)



In [246]:
# Normalizar nombres de columnas elimina eapcios en blanco y convierte a minúsculas
df1.columns = df1.columns.str.lower().str.strip()

In [247]:
# Direcciones y teléfonos faltantes
df1['direccion'] = df1['direccion'].fillna('SIN_DIRECCION')
df1['telefono']  = df1['telefono'].fillna('SIN_TELEFONO')
df1['director']  = df1['director'].fillna('SIN_DIRECTOR')

In [248]:
# Eliminar duplicados
df1 = df1.drop_duplicates()

In [249]:
# Unificar mayúsculas minúsculas y espacios
text_cols = ['distrito','departamento','municipio','establecimiento',
             'direccion','supervisor','director','sector','area',
             'modalidad','jornada','plan','departamental']
for c in text_cols:
    df1[c] = (df1[c]
              .astype(str)
              .str.strip()
              .str.lower()
              .str.replace(r'\s+', ' ', regex=True))

### Limpieza Telefono variable

In [250]:
# Limpieza de teléfonos
import re

def clean_phone(x):
    x = str(x)  # Convertimos a string
    nums = re.sub(r'\D', '', x)
    return nums if len(nums) >= 8 else 'FORMATO_INV'

df1['telefono'] = df1['telefono'].apply(clean_phone)


En la variable telefono se realiza una limpieza que consiste en eliminar todos los caracteres no numéricos (como espacios, guiones, paréntesis o letras), dejando únicamente los dígitos. Si el resultado tiene menos de 8 dígitos, se considera un formato inválido y se reemplaza por 'FORMATO_INV'. Esto estandariza los números de teléfono y facilita su validación

In [251]:
# Conteo de formatos inválidos de teléfono
df1['telefono'].value_counts()['FORMATO_INV']



68

In [252]:
# ver lasfilas con formato inválido
df1[df1['telefono'] == 'FORMATO_INV'].head()

,codigo,distrito,departamento,municipio,establecimiento,direccion,telefono,supervisor,director,sector,area,modalidad,jornada,plan,departamental
83,16-03-0036-46,16-053,alta verapaz,san cristobal verapaz,instituto nacional de educacion diversificada,barrio chiyuc,FORMATO_INV,leonardo oxom chen,edy rolando jom coy,oficial,rural,monolingue,vespertina,diario(regular),alta verapaz
136,16-06-2398-46,16-011,alta verapaz,san miguel tucuru,instituto privado mixto de magisterio bilingue...,canton la playa,FORMATO_INV,ingrid amanda camó tobar de castro,miguel fernando tut lemus,privado,urbana,monolingue,vespertina,diario(regular),alta verapaz
151,16-07-0253-46,16-055,alta verapaz,panzos,colegio privado mixto juan amos comenio,colonia santa isabel,FORMATO_INV,maria elena palma molina,marcelino caz mucú,privado,rural,monolingue,doble,fin de semana,alta verapaz
170,16-09-0007-46,16-033,alta verapaz,san pedro carcha,centro educativo nueva vida,8a. calle 4-47 zona 4,FORMATO_INV,oscar rené ventura zac,juan rax,privado,rural,monolingue,vespertina,diario(regular),alta verapaz
207,16-09-9259-46,16-033,alta verapaz,san pedro carcha,"colegio ""sthella de hernandez""",diagonal 2 10-73 zona 2 barrio san pablo,FORMATO_INV,oscar rené ventura zac,orley edwin meléndez rosales,privado,urbana,monolingue,doble,fin de semana,alta verapaz


### Limpieza Establecimiento variable

In [253]:
# Eliminar tildes, comillas y caracteres especiales de establecimiento
df1['establecimiento'] = df1['establecimiento'].apply(lambda x: 
    re.sub(r'[^\w\s]', '',  
        unicodedata.normalize('NFD', x)  
        .encode('ascii', 'ignore')       
        .decode('utf-8')                 
    )
)

A la variable establecimiento se le aplicó una limpieza textual que incluyó: la conversión a minúsculas, eliminación de espacios duplicados y al inicio/final, eliminación de tildes y acentos mediante normalización Unicode, y la eliminación de signos de puntuación y caracteres especiales como comillas. Esto garantiza que los nombres de los establecimientos queden en un formato uniforme y comparable, reduciendo errores por variaciones tipográficas.

### Limpieza Direccion variable

In [254]:
df1.head(5)

,codigo,distrito,departamento,municipio,establecimiento,direccion,telefono,supervisor,director,sector,area,modalidad,jornada,plan,departamental
0,16-01-0138-46,16-031,alta verapaz,coban,colegio coban,km.2 salida a san juan chamelco zona 8,77945104,patricio najarro asencio,gustavo adolfo sierra pop,privado,urbana,monolingue,matutina,diario(regular),alta verapaz
1,16-01-0139-46,16-031,alta verapaz,coban,colegio particular mixto verapaz,km 209.5 entrada a la ciudad,77367402,patricio najarro asencio,gilma dolores guay paz de leal,privado,urbana,monolingue,matutina,diario(regular),alta verapaz
2,16-01-0140-46,16-031,alta verapaz,coban,colegio la inmaculada,7a. avenida 11-109 zona 6,78232301,patricio najarro asencio,virginia solano serrano,privado,urbana,monolingue,matutina,diario(regular),alta verapaz
3,16-01-0141-46,16-005,alta verapaz,coban,escuela nacional de ciencias comerciales,2a calle 11-10 zona 2,79514215,nora liliana figueroa hernández,héctor rolando chun poou,oficial,urbana,monolingue,matutina,diario(regular),alta verapaz
4,16-01-0142-46,16-005,alta verapaz,coban,instituto normal mixto del norte emilio rosale...,3a ave 6-23 zona 11,79521468,nora liliana figueroa hernández,victor hugo domínguez reyes,oficial,urbana,bilingue,vespertina,diario(regular),alta verapaz


In [255]:
# Quitar tildes acentos
df1['direccion'] = df1['direccion'].apply(
    lambda x: unicodedata.normalize('NFD', x).encode('ascii', 'ignore').decode('utf-8')
)

# Eliminar comillas y símbolos especiales innecesarios (se conserva punto y guion)
df1['direccion'] = df1['direccion'].str.replace(r'[\"\'“”]', '', regex=True)
df1['direccion'] = df1['direccion'].str.replace(r'[^\w\s\.\-]', '', regex=True)

# poner abreviaturas comunes
df1['direccion'] = df1['direccion'].str.replace(r'\bavenida\b', 'av', regex=True).str.replace(r'\bzona\b', 'z', regex=True)


En la variable direccion se aplicó una limpieza para mejorar la consistencia del texto sin perder su significado. Se eliminaron tildes y acentos mediante normalización Unicode, y se retiraron comillas y símbolos especiales innecesarios, conservando únicamente letras, números, puntos y guiones. Además, se estandarizaron términos comunes reemplazando la palabra completa "avenida" por "av" y "zona" por "z", lo cual facilita el análisis posterior.


In [256]:
# Verificar filas completamente duplicadas
duplicados_exactos = df1.duplicated()
print("Total de filas duplicadas exactas:", duplicados_exactos.sum())

# Ver detalles si hay duplicados
df1[duplicados_exactos].head()


Total de filas duplicadas exactas: 0


,codigo,distrito,departamento,municipio,establecimiento,direccion,telefono,supervisor,director,sector,area,modalidad,jornada,plan,departamental


In [257]:
# Duplicados exactos por establecimiento, dirección, municipio y departamento
duplicados_estab_dir_mun_dep = df1.duplicated(subset=['establecimiento', 'direccion', 'municipio', 'departamento'], keep=False)

# Mostrar las filas duplicadas ordenadas por esas columnas
df1[duplicados_estab_dir_mun_dep].sort_values(by=['establecimiento', 'direccion', 'municipio', 'departamento']).head(10)


,codigo,distrito,departamento,municipio,establecimiento,direccion,telefono,supervisor,director,sector,area,modalidad,jornada,plan,departamental
2374,01-08-0346-46,01-603,guatemala,mixco,academia comercial e instituto lourdes,5a. av 6-51 a z 1,55184534,rosa albelia ardon de motta,esteban tocon xicay,privado,urbana,monolingue,doble,fin de semana,guatemala occidente
2375,01-08-0348-46,01-603,guatemala,mixco,academia comercial e instituto lourdes,5a. av 6-51 a z 1,55184534,rosa albelia ardon de motta,esteban tocon xicay,privado,urbana,monolingue,doble,fin de semana,guatemala occidente
2496,01-08-0869-46,01-603,guatemala,mixco,academia comercial e instituto lourdes,5a. av 6-51 a z 1,55319926,rosa albelia ardon de motta,esteban tocon xicay,privado,urbana,monolingue,sin jornada,semipresencial (fin de semana),guatemala occidente
862,00-01-0281-46,01-316,ciudad capital,zona 1,centro cultural de las americas no 3,2a. av 12-23,22321894,carlos humberto gonzalez de leon,brenda paola gonzález recinos,privado,urbana,monolingue,vespertina,diario(regular),guatemala norte
867,00-01-0343-46,01-403,ciudad capital,zona 1,centro cultural de las americas no 3,2a. av 12-23,22325526,carlos humberto gonzález de león,hugo valdemar rivas morales,privado,urbana,monolingue,doble,fin de semana,guatemala norte
1679,00-21-0095-46,01-635,ciudad capital,zona 21,centro de aprendizaje a distancia educativo cade,8a. calle 10-30 colonia vasquez,57044487,nataly fabiola soto españa de turcios,marco tulio roblero osorio,privado,urbana,monolingue,matutina,a distancia,guatemala sur
1680,00-21-0096-46,01-641,ciudad capital,zona 21,centro de aprendizaje a distancia educativo cade,8a. calle 10-30 colonia vasquez,57044487,milton alonso alvarez fuentes,marco tulio roblero osorio,privado,urbana,monolingue,sin jornada,a distancia,guatemala sur
2540,01-08-0995-46,01-639,guatemala,mixco,centro de aprendizaje isos,14 av 0-81 urbanizacion gonzalez z 2 mixco,39914060,ludvin ricardo urrutia lorenti,dilia morales hernández,privado,urbana,monolingue,sin jornada,a distancia,guatemala occidente
2541,01-08-0996-46,01-639,guatemala,mixco,centro de aprendizaje isos,14 av 0-81 urbanizacion gonzalez z 2 mixco,39914060,ludvin ricardo urrutia lorenti,dilia morales hernández,privado,urbana,monolingue,doble,a distancia,guatemala occidente
2532,01-08-0964-46,01-639,guatemala,mixco,centro de aprendizajes isos,14 av 0-81 urbanizacion gonzalez z 2,39914060,ludvin ricardo urrutia lorenti,dilia morales hernández,privado,urbana,monolingue,sin jornada,semipresencial (fin de semana),guatemala occidente


In [ ]:
# Definir las columnas clave
columnas_clave = ['establecimiento', 'direccion', 'municipio', 'departamento','departamental',]

# Separar duplicados de únicos
mask = df1.duplicated(subset=columnas_clave, keep=False)
dups  = df1[mask]
uniqs = df1[~mask]

# Agrupar duplicados y combinar el resto de columnas en listas
agg_dict = {col: list for col in df1.columns if col not in columnas_clave}
df_dups_agg = dups.groupby(columnas_clave, as_index=False).agg(agg_dict)

# Reconstruir el DataFrame final uniendo únicos y agrupados
df_pivot = pd.concat([uniqs, df_dups_agg], ignore_index=True)


En este fragmento de código se realiza un proceso para identificar registros duplicados en un DataFrame df1 basándose en un conjunto de columnas clave (establecimiento, direccion, municipio, departamento, departamental). Primero, se separan los registros duplicados de los únicos. Luego, los duplicados se agrupan por las columnas clave y el resto de las columnas se agregan como listas para conservar toda la información. Finalmente, se reconstruye un nuevo DataFrame (df_pivot) que contiene tanto los registros únicos como los duplicados agrupados, dejando el conjunto de datos más consolidado y sin repeticiones redundantes.

In [270]:
df_pivot[duplicados_estab_dir_mun_dep].sort_values(by=['establecimiento', 'direccion', 'municipio', 'departamento', 'departamental']).head(10)


C:\Users\andre\AppData\Local\Temp\ipykernel_20924\83308012.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_pivot[duplicados_estab_dir_mun_dep].sort_values(by=['establecimiento', 'direccion', 'municipio', 'departamento', 'departamental']).head(10)


,codigo,distrito,departamento,municipio,establecimiento,direccion,telefono,supervisor,director,sector,area,modalidad,jornada,plan,departamental
3793,"[01-08-0346-46, 01-08-0348-46, 01-08-0869-46]","[01-603, 01-603, 01-603]",guatemala,mixco,academia comercial e instituto lourdes,5a. av 6-51 a z 1,"[55184534, 55184534, 55319926]","[rosa albelia ardon de motta, rosa albelia ard...","[esteban tocon xicay, esteban tocon xicay, est...","[privado, privado, privado]","[urbana, urbana, urbana]","[monolingue, monolingue, monolingue]","[doble, doble, sin jornada]","[fin de semana, fin de semana, semipresencial ...",guatemala occidente
789,00-13-7170-46,01-501,ciudad capital,zona 13,academia de estudios avanzados f15,15 av a 24-08,23322290,julio jacobo gil urbina,alma ileana barrios barrios,privado,urbana,monolingue,matutina,diario(regular),guatemala oriente
2025,13-26-0236-46,13-043,huehuetenango,santa cruz barillas,asociacion de maestros de educacion rural de g...,2a. calle 6-35 z 6,30429444,aracely figueroa urbina,alberto efraín ramón diego,privado,urbana,monolingue,sin jornada,semipresencial (fin de semana),huehuetenango
3144,03-01-0194-46,03-016,sacatepequez,antigua guatemala,ceex escuela de caficultura antigua cofee,hacienda finca carmona aldea san juan del obispo,42176394,oscar oswaldo son camez,marvin eduardo gonzález carcuz,privado,rural,monolingue,doble,fin de semana,sacatepéquez
2787,09-20-0126-46,09-012,quetzaltenango,coatepeque,ceexcoatepeque,3ra av 5-00 barrio san francisco,34988123,juan jose colop colop,marta virginia barrios sánchez,privado,urbana,monolingue,doble,fin de semana,quetzaltenango
206,15-03-0778-46,15-031,baja verapaz,rabinal,centro cultural de america,3 calle 2-87 z 2,79388718,edgar ovidio sic xitumul,fabio raxcaco ismalej,privado,urbana,monolingue,vespertina,diario(regular),baja verapaz
207,15-03-1888-46,15-031,baja verapaz,rabinal,centro cultural de america,3era. calle 2-87 z 2,79388718,edgar ovidio sic xitumul,fabio raxcaco ismalej,privado,urbana,monolingue,vespertina,diario(regular),baja verapaz
1853,01-17-8771-46,01-204,guatemala,san miguel petapa,centro cultural sagrada familia no 1,lote 4 manzana a sector 2 villahermosa ii,24484100,alfredo noheli temaj morales,gloria estela guerra gutiérrez,privado,urbana,monolingue,matutina,diario(regular),guatemala sur
3795,"[00-21-0095-46, 00-21-0096-46]","[01-635, 01-641]",ciudad capital,zona 21,centro de aprendizaje a distancia educativo cade,8a. calle 10-30 colonia vasquez,"[57044487, 57044487]","[nataly fabiola soto españa de turcios, milton...","[marco tulio roblero osorio, marco tulio roble...","[privado, privado]","[urbana, urbana]","[monolingue, monolingue]","[matutina, sin jornada]","[a distancia, a distancia]",guatemala sur
1371,01-08-0548-46,01-609,guatemala,mixco,centro de aprendizaje bilingue no 1,7a. calle 3-96 z 8 balcones ii san cristobal,25048292,silvia hortensia gonzalez martinez,gryselda ramirez barrientos,privado,urbana,monolingue,doble,diario(regular),guatemala occidente


In [ ]:
df_expanded.to_csv('nombre_del_archivo.csv', index=False)
